In [6]:
from cred import GUARDIAN_KEY
import requests
import pandas as pd

In [4]:
API_ENDPOINT = 'http://content.guardianapis.com/search'

In [14]:
query = '"royal family"'
page_size = 50
section = 'uk-news'

In [15]:
params = {'api-key':GUARDIAN_KEY,
          'q':query,
          'page-size':page_size,
          'show-tags':'keyword',
          'tag':'type/article',
          'order-by':'newest',
          'page':1,
          'section':section
          }

In [16]:
res = requests.get(API_ENDPOINT, params=params)

In [17]:
res.data

AttributeError: 'Response' object has no attribute 'data'